In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
import dynamo

# function from pyrovelocity
def load_larry(
    file_path: str = "data/external/larry.h5ad",
) -> anndata.AnnData:
    """In vitro Hemotopoiesis Larry datasets

    Data from `CALEB WEINREB et al. (2020) <DOI: 10.1126/science.aaw3381>'
    https://figshare.com/ndownloader/articles/20780344/versions/1

    Returns
    -------
    Returns `adata` object
    """
    url = "https://figshare.com/ndownloader/files/37028569"
    adata = sc.read(file_path, backup_url=url, sparse=True, cache=True)
    return adata


seed = 0
np.random.seed(seed)

/home/mvinyard/anaconda3/envs/dynamo/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mvinyard/anaconda3/envs/dynamo/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mvinyard/anaconda3/envs/dynamo/lib/python3.9/site-packages/uma

In [2]:
adata = load_larry()
adata

AnnData object with n_obs × n_vars = 49302 × 23420
    obs: 'Library', 'Cell barcode', 'time_info', 'Starting population', 'state_info', 'Well', 'SPRING-x', 'SPRING-y'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    uns: 'data_des'
    obsm: 'X_clone', 'X_emb'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

### Subset adata for training

In [3]:
adata_train = adata[adata.obs['Well'].isin([0, 1])].copy()

In [4]:
dynamo.pp.recipe_monocle(adata_train)

|-----? dynamo.preprocessing.deprecated is deprecated.
|-----> recipe_monocle_keep_filtered_cells_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_cells_key=True
|-----> recipe_monocle_keep_filtered_genes_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_genes_key=True
|-----> recipe_monocle_keep_raw_layers_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_raw_layers_key=True
|-----> apply Monocole recipe to adata...


/tmp/ipykernel_2092/1350339928.py:1: DeprecationWarning: recipe_monocle is deprecated and will be removed in a future release. Please update your code to use the new replacement function.
  dynamo.pp.recipe_monocle(adata_train)


|-----> ensure all cell and variable names unique.
|-----> ensure all data in different layers in csr sparse matrix format.
|-----> ensure all labeling data properly collapased
|-----> filtering cells...
|-----> 23491 cells passed basic filters.
|-----> filtering gene...
|-----> 5244 genes passed basic filters.
|-----> calculating size factor...
|-----> selecting genes in layer: X, sort method: SVR...
|-----> size factor normalizing the data, followed by log1p transformation.
|-----> Set <adata.X> to normalized data
|-----> applying PCA ...
|-----> <insert> X_pca to obsm in AnnData Object.
|-----> cell cycle scoring...
|-----> computing cell phase...
|-----> [Cell Phase Estimation] completed [38.7698s]
|-----> [Cell Cycle Scores Estimation] completed [10.9589s]
|-----> [recipe_monocle preprocess] completed [31.1441s]


In [5]:
dynamo.tl.dynamics(adata_train, model='stochastic')

|-----> dynamics_del_2nd_moments_key is None. Using default value from DynamoAdataConfig: dynamics_del_2nd_moments_key=False
|-----------> removing existing M layers:[]...
|-----------> making adata smooth...
|-----> calculating first/second moments...
|-----> [moments calculation] completed [73.0941s]


estimating gamma: 100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [13:56<00:00,  2.39it/s]


AnnData object with n_obs × n_vars = 23697 × 23420
    obs: 'Library', 'Cell barcode', 'time_info', 'Starting population', 'state_info', 'Well', 'SPRING-x', 'SPRING-y', 'nGenes', 'nCounts', 'pMito', 'pass_basic_filter', 'Size_Factor', 'initial_cell_size', 'unspliced_Size_Factor', 'initial_unspliced_cell_size', 'spliced_Size_Factor', 'initial_spliced_cell_size', 'ntr', 'cell_cycle_phase'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'nCells', 'nCounts', 'pass_basic_filter', 'log_m', 'score', 'log_cv', 'frac', 'use_for_pca', 'ntr', 'beta', 'gamma', 'half_life', 'alpha_b', 'alpha_r2', 'gamma_b', 'gamma_r2', 'gamma_logLL', 'delta_b', 'delta_r2', 'bs', 'bf', 'uu0', 'ul0', 'su0', 'sl0', 'U0', 'S0', 'total0', 'use_for_dynamics'
    uns: 'data_des', 'pp', 'velocyto_SVR', 'PCs', 'explained_variance_ratio_', 'pca_mean', 'pca_fit', 'feature_selection', 'cell_phase_genes', 'dynamics'
    obsm: 'X_clone', 'X_emb', 'X_pca', 'X', 'cell_cycle_scores'
    layers: 'ambiguous', 'matrix', 

In [6]:
dynamo.tl.reduceDimension(adata_train)

|-----> retrieve data for non-linear dimension reduction...
|-----> [UMAP] using X_pca with n_pca_components = 30
|-----> <insert> X_umap to obsm in AnnData Object.
|-----> [UMAP] completed [47.3075s]


In [7]:
dynamo.tl.cell_velocities(adata_train, method='pearson', basis='pca', other_kernels_dict={'transform': 'sqrt'})

|-----> incomplete neighbor graph info detected: connectivities and distances do not exist in adata.obsp, indices not in adata.uns.neighbors.
|-----> Neighbor graph is broken, recomputing....
|-----> Start computing neighbor graph...
|-----------> X_data is None, fetching or recomputing...
|-----> fetching X data from layer:None, basis:pca
|-----> method arg is None, choosing methods automatically...
|-----------> method ball_tree selected
|-----> [calculating transition matrix via pearson kernel with sqrt transform.] in progress: 100.0000%|-----> [calculating transition matrix via pearson kernel with sqrt transform.] completed [65.9141s]
|-----> [projecting velocity vector to low dimensional embedding] in progress: 100.0000%|-----> [projecting velocity vector to low dimensional embedding] completed [20.2797s]


AnnData object with n_obs × n_vars = 23697 × 23420
    obs: 'Library', 'Cell barcode', 'time_info', 'Starting population', 'state_info', 'Well', 'SPRING-x', 'SPRING-y', 'nGenes', 'nCounts', 'pMito', 'pass_basic_filter', 'Size_Factor', 'initial_cell_size', 'unspliced_Size_Factor', 'initial_unspliced_cell_size', 'spliced_Size_Factor', 'initial_spliced_cell_size', 'ntr', 'cell_cycle_phase'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'nCells', 'nCounts', 'pass_basic_filter', 'log_m', 'score', 'log_cv', 'frac', 'use_for_pca', 'ntr', 'beta', 'gamma', 'half_life', 'alpha_b', 'alpha_r2', 'gamma_b', 'gamma_r2', 'gamma_logLL', 'delta_b', 'delta_r2', 'bs', 'bf', 'uu0', 'ul0', 'su0', 'sl0', 'U0', 'S0', 'total0', 'use_for_dynamics', 'use_for_transition'
    uns: 'data_des', 'pp', 'velocyto_SVR', 'PCs', 'explained_variance_ratio_', 'pca_mean', 'pca_fit', 'feature_selection', 'cell_phase_genes', 'dynamics', 'neighbors', 'umap_fit', 'grid_velocity_pca'
    obsm: 'X_clone', 'X_emb', '

In [8]:
dynamo.vf.VectorField(adata_train, basis='pca', M=1000)

|-----> VectorField reconstruction begins...
|-----> Retrieve X and V based on basis: PCA. 
        Vector field will be learned in the PCA space.
|-----> Learning vector field with method: sparsevfc.
|-----> [SparseVFC] begins...
|-----> Sampling control points based on data velocity magnitude...
|-----> [SparseVFC] completed [6.8910s]
|-----> [VectorField] completed [8.5498s]


In [25]:
idx_df = pd.read_csv("idx_barcode_df.csv", index_col = 0)
idx_values = idx_df['barcode'].tolist()
idx_df

,barcode
13199,d2_3:GCGCTGATTAGCCTCG
13210,d2_3:CGTTGCCTCACAGTTT
13226,d2_3:TCGTGGGTCGTCAGCA
13235,d2_3:TGCGACTATAACCCGT
13256,d2_3:GTTGTCCGTAACCATC
...,...
69313,LK_d2:CGTGTGTTACGCAGAG
69317,LK_d2:GGTTACACCCGCAACT
69322,LK_d2:GCCTGGTATATTGCCT
69324,LK_d2:GGGTCATTTGCGTATC


In [26]:
fate_adata = dynamo.pd.fate(
    adata_train,
    init_cells=idx_values,
    basis='pca',
    direction='forward',
)

uniformly sampling points along a trajectory: 100%|████████████████████████████████████████████████| 2081/2081 [00:17<00:00, 119.71it/s]


In [31]:
fate_output = dynamo.pd.fate_bias(fate_adata, 'state_info', basis='pca',seed=seed)

calculating fate distributions: 2081it [04:23,  7.90it/s]


In [38]:
fate_output.to_csv(path_or_buf=f'dynamo_fatebias.seed_{seed}.csv')

In [48]:
fate_output

,confidence,Baso,Ccr7_DC,Eos,Erythroid,Lymphoid,Mast,Meg,Monocyte,Neutrophil,Undifferentiated,pDC
d2_3:GCGCTGATTAGCCTCG,0.004505,0.000000,0.0,0.0,0.000000,0.079487,0.0,0.0,0.000000,0.000000,0.920513,0.0
d2_3:CGTTGCCTCACAGTTT,0.004505,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.0
d2_3:TCGTGGGTCGTCAGCA,0.004673,0.000000,0.0,0.0,0.228205,0.000000,0.0,0.0,0.000000,0.000000,0.771795,0.0
d2_3:TGCGACTATAACCCGT,0.004608,0.000000,0.0,0.0,0.325641,0.000000,0.0,0.0,0.000000,0.000000,0.674359,0.0
d2_3:GTTGTCCGTAACCATC,0.006897,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
LK_d2:CGTGTGTTACGCAGAG,0.006494,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.0
LK_d2:GGTTACACCCGCAACT,0.004464,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.0
LK_d2:GCCTGGTATATTGCCT,0.004329,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.112821,0.115385,0.771795,0.0
LK_d2:GGGTCATTTGCGTATC,0.004739,0.548718,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.451282,0.0


In [41]:
fates = ['Undifferentiated', 'Ccr7_DC', 'Eos', 'Erythroid', 'Lymphoid', 'Mast', 'Meg', 'Monocyte', 'Neutrophil', 'pDC']

In [44]:
fate_output[fates].idxmax(1).value_counts()

Undifferentiated    1826
Meg                  161
Monocyte              67
Mast                  25
Neutrophil             2
Name: count, dtype: int64